# DehazeNet

In [5]:
#/content

In [10]:
!wget http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip

--2021-02-11 18:10:43--  http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip
Resolving www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)... 129.132.19.216, 2001:67c:10ec:254::216
Connecting to www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)|129.132.19.216|:80... connected.
HTTP request sent, awaiting response... 301 
Location: https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip [following]
--2021-02-11 18:10:43--  https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573320339 (547M) [application/zip]
Saving to: ‘O-HAZE.zip’

O-HAZE.zip          100%[===================>] 546.76M  10.9MB/s    in 53s     

2021-02-11 18:11:37 (10.4 MB/s) - ‘O-HAZE.zip’ saved [573320339/573320339]



In [13]:
!unzip O-HAZE.zip

Archive:  O-HAZE.zip
   creating: # O-HAZY NTIRE 2018/
   creating: # O-HAZY NTIRE 2018/GT/
 extracting: # O-HAZY NTIRE 2018/GT/01_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/02_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/03_outdoor_GT.JPG  
  inflating: # O-HAZY NTIRE 2018/GT/04_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/05_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/06_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/07_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/08_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/09_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/10_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/11_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/12_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/13_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/14_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/15_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/16_outdoor_GT.jpg  
 extracting:

mv: missing file operand
Try 'mv --help' for more information.


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam
BATCH_SIZE = 128
EPOCH = 10


In [5]:

class DehazeNet(Model):
  def __init__(self, inputs=16, groups=4):
    super(DehazeNet, self).__init__()
    self.inputs = inputs
    self.groups = groups
    self.conv1=tf.keras.layers.Conv2D(16,kernel_size=5)
    self.conv2=tf.keras.layers.Conv2D(16,kernel_size=3,padding='SAME')
    self.conv3=tf.keras.layers.Conv2D(16,kernel_size=5,padding='SAME')
    self.conv4=tf.keras.layers.Conv2D(16,kernel_size=7,padding='SAME')
    self.maxpool=tf.keras.layers.MaxPool2D(7,1)
    self.conv5=tf.keras.layers.Conv2D(1,kernel_size=6)
    self.brelu=tf.keras.activations.relu

  def Maxout(self,x,groups):
    x = x.reshape(x.shape[0], groups, x.shape[1]//groups, x.shape[2], x.shape[3])
    x, y = np.max(x, dim=2, keepdim=True)
    out = x.reshape(x.shape[0],-1, x.shape[3], x.shape[4])
    return out

  def call(self, x):
    out = self.conv1(x)
    out = self.Maxout(out, self.groups)
    out1 = self.conv2(out)
    out2 = self.conv3(out)
    out3 = self.conv4(out)
    y = tf.concat((out1,out2,out3), dim=1)
		#print(y.shape[0],y.shape[1],y.shape[2],y.shape[3],)
    y = self.maxpool(y)
		#print(y.shape[0],y.shape[1],y.shape[2],y.shape[3],)
    y = self.conv5(y)
		# y = self.relu(y)
		# y = self.BRelu(y)
		#y = torch.min(y, torch.ones(y.shape[0],y.shape[1],y.shape[2],y.shape[3]))
    y = self.brelu(y)
    y = y.reshape(y.shape[0],-1)
    return y

In [6]:
model=DehazeNet()

In [9]:
model.build(input_shape=(10, 480, 480, 3))
#model.summary()

In [20]:
def data_path(orig_img_path, hazy_img_path):
    
    train_img = []
    val_img = []
    
    orig_img = glob.glob('./# O-HAZY NTIRE 2018/GT/*.jpg')        
    hazy_img = glob.glob('./# O-HAZY NTIRE 2018/GT/*.jpg')
    
    for x in range(45):
        if (x < 40):
            train_img.append([hazy_img[x], orig_img[x]])
        else:
            val_img.append([hazy_img[x], orig_img[x]])
            
    return train_img, val_img

epochs = 1
batch_size = 3

train_data, val_data = data_path(orig_img_path = './# O-HAZY NTIRE 2018/GT', 
                                 hazy_img_path = './# O-HAZY NTIRE 2018/hazy')


IndexError: ignored

In [17]:
ls

 O-HAZE.zip  '# O-HAZY NTIRE 2018'/   sample_data/


In [19]:
ls ./# O-HAZY NTIRE 2018/GT


ls: cannot access './#': No such file or directory
ls: cannot access 'O-HAZY': No such file or directory
ls: cannot access 'NTIRE': No such file or directory
ls: cannot access '2018/GT': No such file or directory
